# Whale/Dolphin Transfer Learning3
https://www.kaggle.com/stpeteishii/whale-dolphin-transfer-learning3

In [ ]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt 
tf.__version__

# Preprocessing with ImageDataGenerator

## Prepare ImageDataGenerator
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [ ]:
img_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                            #rotation_range=90,
                            brightness_range=(0.5,1), 
                            #shear_range=0.2, 
                            #zoom_range=0.2,
                            channel_shift_range=0.2,
                            horizontal_flip=False,
                            vertical_flip=False,
                            rescale=1./255,
                            validation_split=0.3)

## img_generator.flow_from_dataframe

In [ ]:
train_dir='../input/happy-whale-and-dolphin/train_images'
train=pd.read_csv('../input/happy-whale-and-dolphin/train.csv')
train2=train[['image','species']].copy()
train2['path']=train2['image'].apply(lambda x: os.path.join(train_dir,x))

In [ ]:
train2

In [ ]:
Name0=train2['species'].unique().tolist()
Name=sorted(Name0)
print(len(Name))
print(Name)
N=list(range(len(Name)))
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [ ]:
#train2['species']=train2['species'].map(normal_mapping)

train3 = pd.DataFrame().assign(ImagePath=train2.loc[:,'path'], ImageClass=train2.loc[:,'species'])
train3

In [ ]:
test_dir='../input/happy-whale-and-dolphin/test_images'
test=pd.read_csv('../input/happy-whale-and-dolphin/sample_submission.csv')
test2=test.copy()
test2['path']=test2['image'].apply(lambda x: os.path.join(test_dir,x))

In [ ]:
test2['species']='Tiger'  ### dummy data

test3 = pd.DataFrame().assign(ImagePath=test2.loc[:,'path'], ImageClass=test2.loc[:,'species'])
test3

In [ ]:
img_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

timg_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
#img_generator_flow_train = img_generator.flow_from_directory
#img_generator_flow_train = img_generator.flow_from_dataframe

img_generator_flow_train = img_generator.flow_from_dataframe(
    train3,
    x_col='ImagePath',
    y_col='ImageClass',
    directory='',
    target_size=(224, 224),
    batch_size=64,
    shuffle=True,
    subset="training",
    class_mode='categorical',
)

img_generator_flow_valid = img_generator.flow_from_dataframe(
    train3,
    x_col='ImagePath',
    y_col='ImageClass',
    directory='',
    target_size=(224, 224),
    batch_size=64,
    shuffle=True,
    subset="validation",
    class_mode='categorical',
)


## Visualize a batch of images

In [ ]:
imgs, labels = next(iter(img_generator_flow_train))
for img, label in zip(imgs, labels):
    value=np.argmax(label)
    plt.imshow(img)
    plt.title('Species: '+reverse_mapping[value])
    plt.axis("off")
    plt.show()

# Transfer Learning 

## Import a pretrained model
https://www.tensorflow.org/api_docs/python/tf/keras/applications/InceptionV3

In [ ]:
base_model = tf.keras.applications.InceptionV3(input_shape=(224,224,3),
                                               include_top=False,
                                               weights = "imagenet"
                                               )

## Set the weights of the imported model

In [ ]:
base_model.trainable = False

## Create model

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(len(Name), activation="softmax")
])

In [ ]:
model.summary()

## Compile model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = [tf.keras.metrics.CategoricalAccuracy()])

## Train the model

In [ ]:
model.fit(img_generator_flow_train, 
          validation_data=img_generator_flow_valid, 
          steps_per_epoch=8, epochs=10)     #####

## Visualize accuracy and loss

In [ ]:
# Visualise train / Valid Accuracy
plt.plot(model.history.history["categorical_accuracy"], c="r", label="train_accuracy")
plt.plot(model.history.history["val_categorical_accuracy"], c="b", label="test_accuracy")
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Visualise train / Valid Loss
plt.plot(model.history.history["loss"], c="r", label="train_loss")
plt.plot(model.history.history["val_loss"], c="b", label="test_loss")
plt.legend(loc="upper left")
plt.show()

# Interpretation with Grad Cam


## Create images and labels

In [ ]:
imgs, labels = next(iter(img_generator_flow_valid))

In [ ]:
print(imgs.shape)
print(labels.shape)

In [ ]:
for layer in model.layers:
    print(layer.name)

In [ ]:
base_model = model.layers[0]

In [ ]:
last_conv_layer_name = "mixed10"
classifier_layer_names = [layer.name for layer in model.layers][1:]

In [ ]:
# We start by setting up the dependencies we will use
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

## make_gradcam_heatmap

In [ ]:
# The Grad-CAM algorithm
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(
    img_array, base_model, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = base_model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(base_model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

## Predict valid data

In [ ]:
# Print what the top predicted class is
preds = model.predict(imgs) ##### 32
pred_labels = tf.argmax(preds, axis=-1) # +1
print(type(pred_labels))
pred_labels2=np.array(pred_labels)
print(type(pred_labels2))
print(pd.DataFrame(pred_labels2).value_counts())

## Create heatmap

In [ ]:
# Generate class activation heatmap
heatmaps = []

for img in imgs:
    heatmap = make_gradcam_heatmap(
    tf.expand_dims(img,axis=0),
        base_model, model, 
        last_conv_layer_name, 
        classifier_layer_names
  )
    heatmaps.append(heatmap)

# Display heatmap
plt.matshow(heatmaps[0])
plt.show()

## Predicted label and heatmap

In [ ]:
from pathlib import Path

for img, pred_label, true_label, heatmap in zip(imgs, pred_labels, labels, heatmaps): 
    # We rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.003 + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    save_path = "saved_img.jpg"
    superimposed_img.save(save_path)
    
    pred_label2=pred_label.numpy()
    true_label2=np.argmax(true_label) # +1

    print("Predicted Species: ",reverse_mapping[pred_label2])
    print("Actual Species: ", reverse_mapping[true_label2])

    display(Image(save_path))

## Classification report

In [ ]:
PRED=pred_labels2.tolist()

In [ ]:
LABEL=[]
for item in labels:   
    LABEL+=[np.argmax(item)]

In [ ]:
print(len(LABEL))
print(len(PRED))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(LABEL,PRED))